In [1]:
import time
import warnings
import numpy as np

warnings.simplefilter("ignore")

import torch
import torchvision
from torch import nn, optim
import torch.multiprocessing as mp
import torchvision.transforms as transforms
from tqdm.notebook import tqdm 


In [2]:
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from opacus.utils.batch_memory_manager import BatchMemoryManager

In [3]:
#CUDA_VISIBLE_DEVICES=0
print(torch.cuda.get_device_name(0))
print(torch.cuda.current_device())
#torch.cuda.set_device(1)

NVIDIA RTX A3000 Laptop GPU
0
0


In [4]:
if torch.cuda.is_available():
    device = "cuda"
    print(torch.cuda.get_device_name(torch.cuda.current_device()))
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
print(device)

NVIDIA RTX A3000 Laptop GPU
cuda


In [5]:
# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [6]:
# Hyperparameters for training and privacy
MAX_GRAD_NORM = 1.0
EPSILON = 5.0
DELTA = 1e-5

EPOCHS = 10
LR = 1e-3

In [7]:
# Function to make the model differentially private
def make_private(model, optimizer, train_loader, epsilon=EPSILON, epochs=EPOCHS):
    model_dp, optimizer_dp, train_loader_dp = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        epochs=epochs,
        target_epsilon=epsilon,
        target_delta=DELTA,
        max_grad_norm=MAX_GRAD_NORM,
    )
    return model_dp, optimizer_dp, train_loader_dp

# Function to calculate accuracy
def accuracy(preds, labels):
    return (preds == labels).mean()

# Function for training the model
def train(model, optimizer, train_loader, device, dp, task, epochs=EPOCHS): 
    start_time = time.time()

    model = model.to(device)

    for epoch in tqdm(range(epochs), desc="Epoch", unit="epoch"):
    
        model.train()
        criterion = nn.CrossEntropyLoss()

        losses = []
        accs = []

        # BatchMemoryManager manages the memory usage (e.g. you can use bigger logical batches)
        with BatchMemoryManager(data_loader=train_loader, max_physical_batch_size=128, optimizer=optimizer) as loader:
            if not dp:
                loader = train_loader

            for i, (images, target) in enumerate(loader):   
                optimizer.zero_grad()
                images = images.to(device)
                target = target.to(device)
                
                if task == 'multi_class': 
                    target = target.squeeze().float()
                
                output = model(images)
                target = target.long()
                loss = criterion(output, target)

                preds = np.argmax(output.detach().cpu().numpy(), axis=1)
                labels = target.detach().cpu().numpy()

                acc = accuracy(preds, labels)

                losses.append(loss.item())
                accs.append(acc)

                loss.backward()
                optimizer.step()

            if dp:
                epsilon = privacy_engine.get_epsilon(DELTA)
            else:
                epsilon = float('inf')
            print(
                f"Training Epoch {epoch+1:02d} \t"
                f"Loss: {np.mean(losses):.6f} | "
                f"Acc: {np.mean(accs) * 100:.6f} | "
                f"ε = {epsilon:.2f}"
            )
                
    end_time = time.time()
    training_time = end_time - start_time
    print(f'\nTraining Time \t\t{training_time:.2f} seconds')
                
# Function for testing the model
def test(model, test_loader, device, dp, task):
    model = model.to(device)
    
    model.eval()
    criterion = nn.CrossEntropyLoss()
        
    losses = []
    accs = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            if task == 'multi_class': 
                target = target.squeeze().float()
                
            output = model(images)
            target = target.long()
            loss = criterion(output, target)
            
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            accs.append(acc)

    print(
        f"Test Set \t\t"
        f"Loss: {np.mean(losses):.6f} | "
        f"Acc: {np.mean(accs) * 100:.6f} "
    )

# Function for the overall pipeline
def pipeline(model, train_loader, test_loader, device, task=None):
    optimizer = optim.RMSprop(model.parameters(), lr=LR)

    # Specific layers of the model can't be made differentially private, so the model has to be fixed
    model_fixed = ModuleValidator.fix(model)
    optimizer_fixed = optim.RMSprop(model_fixed.parameters(), lr=LR)

    model_dp, optimizer_dp, train_loader_dp = make_private(model_fixed, optimizer_fixed, train_loader)
    
    print(f'\n===== With DP =====')
    train(model_dp, optimizer_dp, train_loader_dp, device, True, task)
    test(model_dp, test_loader, device, True, task)
    print(f'\n===== Without DP =====')
    train(model, optimizer, train_loader, device, False, task)
    test(model, test_loader, device, False, task)

In [ ]:
# CIFAR 10
train_dataset = torchvision.datasets.CIFAR10(root='/Users/valentinbiller/Downloads/data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='/Users/valentinbiller/Downloads/data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

# Initialize the privacy engine and the model
privacy_engine = PrivacyEngine()
model = torchvision.models.resnet18(num_classes=10)

pipeline(model, train_loader, test_loader, device)

In [ ]:
import medmnist
from medmnist import INFO

root = '/Users/valentinbiller/Downloads/'
datasets = ['dermamnist', 'pneumoniamnist', 'bloodmnist', 'organcmnist']

transformMedMNIST = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5]),
])

for data_flag in datasets:
    
    info = INFO[data_flag]
    DataClass = getattr(medmnist, info['python_class'])
    
    train_dataset = DataClass(split='train', transform=transformMedMNIST, download=True, root=root)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)

    test_dataset = DataClass(split='test', transform=transformMedMNIST, download=True, root=root)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)
    
    num_classes = len(info['label'])
    
    privacy_engine = PrivacyEngine()
    model = torchvision.models.resnet18(num_classes=num_classes)
    if data_flag == 'pneumoniamnist' or data_flag == 'organcmnist':     
        model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
 
    print(f'\n\n\n ========== {data_flag} ========== \n')
    pipeline(model, train_loader, test_loader, device, task='multi_class')
    print(f'\n\n\n')

## Task 1c

High Privacy (Level 1): Choose a lower target_epsilon value. target_epsilon = 0.1.
Medium Privacy (Level 4): Choose a moderate target_epsilon value. target_epsilon = 1.
Low Privacy (Level 8): Choose a higher target_epsilon value. target_epsilon = 5.

In [8]:
import medmnist
from medmnist import INFO
info = INFO["pneumoniamnist"]
DataClass = getattr(medmnist, info['python_class'])
num_classes = len(info['label'])


In [9]:

datasets = ["pneumoniamnist"] #['dermamnist', 'pneumoniamnist', 'bloodmnist', 'organcmnist']

transformMedMNIST = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5]),
])

# Initialize the privacy engine and the model
privacy_engine = PrivacyEngine()
model = torchvision.models.resnet18(num_classes=10)
epochs = 10
batch_size = 256
lr = 1e-3
  
train_dataset = DataClass(split='train', transform=transformMedMNIST, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = DataClass(split='test', transform=transformMedMNIST, download=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



Using downloaded and verified file: C:\Users\m1choelz\.medmnist\pneumoniamnist.npz
Using downloaded and verified file: C:\Users\m1choelz\.medmnist\pneumoniamnist.npz


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from opacus import PrivacyEngine

# Define the model with adjustable parameters
def build_model(num_layers, dropout_rate):
    model = models.resnet18(pretrained=False)
    
    # Modify the last fully connected layer
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(num_ftrs, 2)  # assuming binary classification
    )
    return model

# Training and evaluation loop
def train_and_evaluate(model, train_loader, test_loader, learning_rate, num_epochs=10, use_dp=False, dp_epsilon=1.0, dp_delta=1e-5):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Attach Privacy Engine to the optimizer if using differential privacy
    if use_dp:
        privacy_engine = PrivacyEngine(
            model,
            batch_size=32,
            sample_size=len(train_loader.dataset),
            alphas=[10, 100],
            noise_multiplier=1.0,
            max_grad_norm=1.0,
        )
        privacy_engine.attach(optimizer)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()  # regular SGD step
        
        # Evaluate the privacy budget spent so far
        if use_dp:
            epsilon, best_alpha = privacy_engine.get_privacy_spent(dp_delta)
            print(f"Train Epoch: {epoch + 1} \t"
                  f"Loss: {loss.item():.6f} \t"
                  f"(ε = {epsilon:.2f}, δ = {dp_delta}) for α = {best_alpha}")
        else:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

    # Evaluation loop
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy} %')
    # Here you would implement a privacy assessment method
    # This could involve differential privacy analysis or other techniques

for num_layers in [18, 34, 50]:  # Different ResNet versions
    for dropout_rate in [0.0, 0.5]:
        for learning_rate in [0.001, 0.0001]:
            print(f"Training with num_layers: {num_layers}, dropout_rate: {dropout_rate}, learning_rate: {learning_rate}")
            model = build_model(num_layers, dropout_rate)
            train_and_evaluate(model, train_loader, test_loader, learning_rate, use_dp=True, num_epochs=10)
            # Add code here to evaluate privacy


In [10]:

model = torchvision.models.resnet18(num_classes=num_classes)    
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model2 = torchvision.models.resnet18(num_classes=num_classes)    
model2.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model3 = torchvision.models.resnet18(num_classes=num_classes)    
model3.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [11]:
# Function for the overall pipeline
from torch.optim import RMSprop, Adam
import copy 
def pipeline1c( train_loader, test_loader, device, task=None, epochs=EPOCHS, optim=(RMSprop), lr=LR):
    level1 = 0.5
    level2 = 4
    level3 = 9

   
    model_fixed = ModuleValidator.fix(model)
    model_fixed2 = ModuleValidator.fix(model2)
    model_fixed3= ModuleValidator.fix(model3)
    optimizer_fixed = optim(model_fixed.parameters(), lr=lr)
    optimizer_fixed2 = optim(model_fixed2.parameters(), lr=lr)
    optimizer_fixed3 = optim(model_fixed3.parameters(), lr=lr)
    model_dp1, optimizer_dp1, train_loader_dp1 = make_private(model_fixed, optimizer_fixed, train_loader, level1, epochs)
    model_dp2, optimizer_dp2, train_loader_dp2 = make_private(model_fixed2, optimizer_fixed2, train_loader, level2,epochs)
    model_dp3, optimizer_dp3, train_loader_dp3 = make_private(model_fixed3, optimizer_fixed3, train_loader,level3,epochs)
    print(f'\n===== With DP level1 =====')
    train(model_dp1, optimizer_dp1, train_loader_dp1, device, True, task,epochs)
    test(model_dp1, test_loader, device, True, task)
    torch.cuda.empty_cache()
    print(f'\n===== With DP level2 =====')
    train(model_dp2, optimizer_dp2, train_loader_dp2, device, True, task,epochs)
    test(model_dp2, test_loader, device, True, task)
    torch.cuda.empty_cache()
    print(f'\n===== With DP level3 =====')
    train(model_dp3, optimizer_dp3, train_loader_dp3, device, True, task,epochs)
    test(model_dp3, test_loader, device, True, task)

In [17]:

batch_size = [64, 128, 256]

for batch_size in batch_size:
    
    privacy_engine = PrivacyEngine()
    # to reset models after training: 
    model = torchvision.models.resnet18(num_classes=num_classes)    
    model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model2 = torchvision.models.resnet18(num_classes=num_classes)    
    model2.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model3 = torchvision.models.resnet18(num_classes=num_classes)    
    model3.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
 
    print(f'\n\n\n ========== {batch_size} ========== \n')
    pipeline1c(train_loader, test_loader, device, task='multi_class', epochs = epochs, optim = RMSprop)
    print(f'\n\n\n')




 ========== 64 ========== 


===== With DP level1 =====

===== With DP level2 =====

===== With DP level3 =====


Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 1.723913 | Acc: 71.840938 | ε = 4.26
Training Epoch 02 	Loss: 0.393171 | Acc: 88.548644 | ε = 5.09
Training Epoch 03 	Loss: 0.208835 | Acc: 94.389611 | ε = 5.75
Training Epoch 04 	Loss: 0.243192 | Acc: 93.968228 | ε = 6.32
Training Epoch 05 	Loss: 0.219192 | Acc: 94.886559 | ε = 6.84
Training Epoch 06 	Loss: 0.193264 | Acc: 95.461154 | ε = 7.32
Training Epoch 07 	Loss: 0.174646 | Acc: 95.575095 | ε = 7.77
Training Epoch 08 	Loss: 0.192411 | Acc: 95.498759 | ε = 8.20
Training Epoch 09 	Loss: 0.205574 | Acc: 95.434538 | ε = 8.60
Training Epoch 10 	Loss: 0.159306 | Acc: 96.217467 | ε = 9.00

Training Time 		6138.37 seconds
Test Set 		Loss: 0.968425 | Acc: 84.114583 






In [9]:

privacy_engine = PrivacyEngine()
model = torchvision.models.resnet34(num_classes=num_classes)    
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model2 = torchvision.models.resnet34(num_classes=num_classes)    
model2.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model3 = torchvision.models.resnet34(num_classes=num_classes)    
model3.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
print(f'\n\n\n ========== resnet34 ========== \n')
pipeline1c( train_loader, test_loader, device, task='multi_class', epochs = 10, optim = RMSprop)
print(f'\n\n\n')




 ========== resnet34 ========== 


===== With DP level1 =====

===== With DP level2 =====

===== With DP level3 =====


Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 2.070636 | Acc: 73.524336 | ε = 4.26
Training Epoch 02 	Loss: 1.265989 | Acc: 74.216246 | ε = 5.09
Training Epoch 03 	Loss: 0.349339 | Acc: 90.628885 | ε = 5.75
Training Epoch 04 	Loss: 0.269412 | Acc: 93.931942 | ε = 6.32
Training Epoch 05 	Loss: 0.238159 | Acc: 94.772765 | ε = 6.84
Training Epoch 06 	Loss: 0.221897 | Acc: 95.097371 | ε = 7.32
Training Epoch 07 	Loss: 0.208748 | Acc: 95.770681 | ε = 7.77
Training Epoch 08 	Loss: 0.216766 | Acc: 95.316235 | ε = 8.20
Training Epoch 09 	Loss: 0.183091 | Acc: 95.815597 | ε = 8.60
Training Epoch 10 	Loss: 0.206137 | Acc: 95.541217 | ε = 9.00

Training Time 		6447.23 seconds
Test Set 		Loss: 1.139144 | Acc: 82.477679 






In [14]:

model = torchvision.models.resnet18(num_classes=num_classes)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

print(f'\n\n\n ========== ADAM ========== \n')
pipeline1c(train_loader, test_loader, device, task='multi_class', epochs = epochs, optim = optim.Adam)
print(f'\n\n\n')




 ========== ADAM ========== 


===== With DP level1 =====

===== With DP level2 =====

===== With DP level3 =====


Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 1.227621 | Acc: 73.764304 | ε = 4.26
Training Epoch 02 	Loss: 0.459083 | Acc: 85.045902 | ε = 5.09
Training Epoch 03 	Loss: 0.287433 | Acc: 92.694988 | ε = 5.75
Training Epoch 04 	Loss: 0.239063 | Acc: 94.283932 | ε = 6.32
Training Epoch 05 	Loss: 0.209389 | Acc: 94.764335 | ε = 6.84
Training Epoch 06 	Loss: 0.179537 | Acc: 95.579621 | ε = 7.32
Training Epoch 07 	Loss: 0.169264 | Acc: 95.490683 | ε = 7.77
Training Epoch 08 	Loss: 0.166808 | Acc: 95.864017 | ε = 8.20
Training Epoch 09 	Loss: 0.168041 | Acc: 96.048149 | ε = 8.60
Training Epoch 10 	Loss: 0.164560 | Acc: 95.803203 | ε = 9.00

Training Time 		2057.86 seconds
Test Set 		Loss: 0.962280 | Acc: 83.258929 






In [12]:
def modify_alexnet(num_classes, input_channels=1):
    # Load AlexNet
    model = models.alexnet(pretrained=False)

    # Modify the first convolution layer for 1-channel input
    model.features[0] = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3)

    # Correct subsequent convolution layers to match the new channel sizes
    model.features[3] = nn.Conv2d(64, 192, kernel_size=5, stride=1, padding=2)   # Second conv layer
    model.features[6] = nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1)  # Third conv layer
    model.features[8] = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)  # Fourth conv layer
    model.features[10] = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1) # Fifth conv layer

    # Modify the max pooling layers to be less aggressive
    model.features[2] = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)   # First max pooling layer
    model.features[5] = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)   # Second max pooling layer
    model.features[12] = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)  # Third max pooling layer

    # Modify the classifier for the number of classes
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

    return model


In [13]:
import torchvision.models as models
import torch.nn as nn
model = modify_alexnet(num_classes) 

model2 = modify_alexnet(num_classes)  

model3 = modify_alexnet(num_classes)   

print(f'\n\n\n ========== alexnet ========== \n')
pipeline1c(train_loader, test_loader, device, task='multi_class', epochs = epochs, optim = RMSprop)
print(f'\n\n\n')




 ========== alexnet ========== 


===== With DP level1 =====


Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

RuntimeError: CUDA out of memory. Tried to allocate 9.00 GiB (GPU 0; 6.00 GiB total capacity; 13.44 GiB already allocated; 0 bytes free; 17.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:

def modify_vgg11(num_classes, input_channels=1):
    # Load VGG11
    model = models.vgg11(pretrained=False)

    # Modify the first convolution layer for 1-channel input
    model.features[0] = nn.Conv2d(input_channels, 64, kernel_size=3, padding=1)

    # Modify the max pooling layers to be less aggressive
    # VGG11 has pooling layers at indices 2, 5, 10, 15, and 20 in the 'features' module
    model.features[2] = nn.MaxPool2d(kernel_size=2, stride=1, padding=0)  # First pooling layer
    model.features[5] = nn.MaxPool2d(kernel_size=2, stride=1, padding=0)  # Second pooling layer
    # Consider adjusting other pooling layers similarly

    # Modify the classifier for the number of classes
    num_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_features, num_classes)

    return model

model = modify_vgg11(num_classes, input_channels=1)  # Use input_channels=3 for RGB images


In [ ]:


print(f'\n\n\n ========== vgg11 ========== \n')
pipeline1c( train_loader, test_loader, device, task='multi_class', epochs = epochs, optim = RMSprop)
print(f'\n\n\n')

In [12]:
lr = [1e-2, 1e-3, 1e-4]

for lr in lr:
    # to reset models after one training round
    model = torchvision.models.resnet18(num_classes=num_classes)    
    model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model2 = torchvision.models.resnet18(num_classes=num_classes)    
    model2.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model3 = torchvision.models.resnet18(num_classes=num_classes)    
    model3.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    info = INFO["pneumoniamnist"]
    DataClass = getattr(medmnist, info['python_class'])
    
    train_dataset = DataClass(split='train', transform=transformMedMNIST, download=True)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = DataClass(split='test', transform=transformMedMNIST, download=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    num_classes = len(info['label'])
    
    privacy_engine = PrivacyEngine()

    print(f'\n\n\n ========== {lr} ========== \n')
    pipeline1c(train_loader, test_loader, device, task='multi_class', epochs = epochs, optim = RMSprop)
    print(f'\n\n\n')

Using downloaded and verified file: C:\Users\m1choelz\.medmnist\pneumoniamnist.npz
Using downloaded and verified file: C:\Users\m1choelz\.medmnist\pneumoniamnist.npz



 ========== 0.0001 ========== 


===== With DP level1 =====


Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 1.335607 | Acc: 71.791420 | ε = 0.16
Training Epoch 02 	Loss: 1.159435 | Acc: 74.026861 | ε = 0.21
Training Epoch 03 	Loss: 0.852616 | Acc: 73.120730 | ε = 0.26
Training Epoch 04 	Loss: 0.622599 | Acc: 74.351145 | ε = 0.31
Training Epoch 05 	Loss: 0.475623 | Acc: 81.056603 | ε = 0.34
Training Epoch 06 	Loss: 0.368885 | Acc: 86.780325 | ε = 0.38
Training Epoch 07 	Loss: 0.271753 | Acc: 90.709477 | ε = 0.41
Training Epoch 08 	Loss: 0.249495 | Acc: 91.586975 | ε = 0.44
Training Epoch 09 	Loss: 0.229783 | Acc: 92.251053 | ε = 0.47
Training Epoch 10 	Loss: 0.239303 | Acc: 92.604443 | ε = 0.50

Training Time 		1652.56 seconds
Test Set 		Loss: 0.752382 | Acc: 80.171131 

===== With DP level2 =====


Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 1.612388 | Acc: 72.685385 | ε = 1.80
Training Epoch 02 	Loss: 0.483016 | Acc: 84.585756 | ε = 2.17
Training Epoch 03 	Loss: 0.240172 | Acc: 93.192696 | ε = 2.48
Training Epoch 04 	Loss: 0.194279 | Acc: 94.871297 | ε = 2.75
Training Epoch 05 	Loss: 0.220001 | Acc: 94.717128 | ε = 3.00
Training Epoch 06 	Loss: 0.226218 | Acc: 94.901062 | ε = 3.23
Training Epoch 07 	Loss: 0.208229 | Acc: 95.027177 | ε = 3.45
Training Epoch 08 	Loss: 0.190505 | Acc: 95.638558 | ε = 3.65
Training Epoch 09 	Loss: 0.188565 | Acc: 95.539665 | ε = 3.85
Training Epoch 10 	Loss: 0.180785 | Acc: 95.567756 | ε = 4.03

Training Time 		3042.72 seconds
Test Set 		Loss: 0.954201 | Acc: 83.072917 

===== With DP level3 =====






In [11]:
MAX_GRAD_NORM = 0.5

print(f'\n\n\n ========== MAX_GRAD_NORM = 0.5 ========== \n')
pipeline1c(train_loader, test_loader, device, task='multi_class', epochs = epochs, optim = RMSprop)
print(f'\n\n\n')




 ========== MAX_GRAD_NORM = 0.5 ========== 


===== With DP level1 =====

===== With DP level2 =====

===== With DP level3 =====


Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 1.782303 | Acc: 72.350231 | ε = 4.26
Training Epoch 02 	Loss: 0.289944 | Acc: 91.913433 | ε = 5.09
Training Epoch 03 	Loss: 0.269135 | Acc: 94.075043 | ε = 5.75
Training Epoch 04 	Loss: 0.234353 | Acc: 94.603674 | ε = 6.32
Training Epoch 05 	Loss: 0.203162 | Acc: 95.371011 | ε = 6.84
Training Epoch 06 	Loss: 0.207649 | Acc: 95.562355 | ε = 7.32
Training Epoch 07 	Loss: 0.197224 | Acc: 95.606469 | ε = 7.77
Training Epoch 08 	Loss: 0.201211 | Acc: 95.567608 | ε = 8.20
Training Epoch 09 	Loss: 0.188878 | Acc: 95.786384 | ε = 8.60
Training Epoch 10 	Loss: 0.209565 | Acc: 95.748765 | ε = 9.00

Training Time 		2923.50 seconds
Test Set 		Loss: 1.113981 | Acc: 83.463542 






In [ ]:
DELTA = 1e-3
privacy_engine = PrivacyEngine()
model = torchvision.models.resnet34(num_classes=num_classes)    
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model2 = torchvision.models.resnet34(num_classes=num_classes)    
model2.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model3 = torchvision.models.resnet34(num_classes=num_classes)    
model3.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

print(f'\n\n\n ========== {DELTA} ========== \n')
pipeline1c(train_loader, test_loader, device, task='multi_class', epochs = epochs, optim = RMSprop)
print(f'\n\n\n')

In [12]:
import torch.nn as nn
import torchvision.models as models

class CustomSqueezeNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomSqueezeNet, self).__init__()
        self.squeezenet = models.squeezenet1_0(pretrained=False)
        self.squeezenet.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=2, padding=0)

        # Add an additional 1x1 convolution to match the channel size
        self.channel_matching_conv = nn.Conv2d(64, 96, kernel_size=1)

        self.squeezenet.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=1)

    def forward(self, x):
        x = self.squeezenet.features[0](x)
        x = self.channel_matching_conv(x)  # Adjust the channels
        x = self.squeezenet.features[1:](x)  # Continue through the rest of the SqueezeNet
        x = self.squeezenet.classifier(x)
        return x.view(x.size(0), -1)

model = CustomSqueezeNet(num_classes)
model3 = CustomSqueezeNet(num_classes)
model2 = CustomSqueezeNet(num_classes)
print(f'\n\n\n ========== squeezenet ========== \n')
pipeline1c(train_loader, test_loader, device, task='multi_class', epochs = epochs, optim = RMSprop)
print(f'\n\n\n')




 ========== squeezenet ========== 


===== With DP level1 =====

===== With DP level2 =====

===== With DP level3 =====


Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 1.486115 | Acc: 53.567863 | ε = 4.26
Training Epoch 02 	Loss: 1.461597 | Acc: 75.143074 | ε = 5.09
Training Epoch 03 	Loss: 1.179804 | Acc: 72.812274 | ε = 5.75
Training Epoch 04 	Loss: 1.026227 | Acc: 73.803154 | ε = 6.32
Training Epoch 05 	Loss: 0.849639 | Acc: 74.915985 | ε = 6.84
Training Epoch 06 	Loss: 0.755139 | Acc: 74.944098 | ε = 7.32
Training Epoch 07 	Loss: 0.747062 | Acc: 76.814854 | ε = 7.77
Training Epoch 08 	Loss: 0.655873 | Acc: 81.749970 | ε = 8.20
Training Epoch 09 	Loss: 0.656698 | Acc: 84.491463 | ε = 8.60
Training Epoch 10 	Loss: 0.760072 | Acc: 85.224640 | ε = 9.00

Training Time 		57.54 seconds
Test Set 		Loss: 0.809665 | Acc: 84.375000 




